In [2]:
#  Importing libraries and module and some setting for notebook

import pandas as pd 
import re
import numpy as np
import time

In [13]:
# Merging Spar and Billa master files
df_billa=pd.read_csv("products_billa.csv")
df_billa = df_billa.rename(columns={"index":"Index"})
df_billa['Index_bl'] ='bl ' + df_billa.index.astype(str)

df_billa=df_billa[['gtin','gtin_variants','Index_bl','name']]

df_billa = df_billa.rename(columns={'name': 'name_bl'})

len(df_billa['gtin'].dropna())

3530

In [4]:
#replacing nan values from gtin column with gtin_variants
for i in range(len(df_billa)):
    if pd.isnull(df_billa['gtin'][i]):
        df_billa['gtin'][i]=df_billa['gtin_variants'][i]
        df_billa['gtin_variants'][i]=np.nan

In [5]:
#dictionary with key = index of df_billa, values = gtin numbers of the product equivalent to index of billa
billaDict={}

for i in range(len(df_billa)):
    if not pd.isnull(df_billa['gtin'][i]):
        temp=df_billa['gtin'][i].split(", ")
        billaDict['bl {}'.format(i)]=temp
    else:
        if not pd.isnull(df_billa['gtin_variants'][i]):
            temp=df_billa['gtin_variants'][i].split(", ")
            billaDict['bl {}'.format(i)]=temp

In [6]:
# Creating a list of lists of [Index, gtin] set
keys = list(billaDict.keys())
billalst=[[]]

for i in range(len(keys)):
    key=keys[i]
    for j in range(len(billaDict[key])):
        value=str(billaDict[key][j])
        billalst.append([key,value])

In [7]:
# creating a dataframe of gtin&index
billaRES = pd.DataFrame(billalst, columns = ['Index_bl','gtin'])
billaRES=billaRES.drop([0])
billaRES=billaRES.reset_index(drop=True)

billaRES

,Index_bl,gtin
0,bl 0,9011900100135
1,bl 0,9011900105161
2,bl 1,9002238270959
3,bl 1,9120106122060
4,bl 1,9002331609946
...,...,...
8911,bl 6744,9019100174802
8912,bl 6746,9009695000631
8913,bl 6746,9009695001461
8914,bl 6747,9010158017684


In [8]:
# add names in bl dataframe
tempbl=df_billa[['Index_bl','name_bl']]
df_bl=pd.merge(left = tempbl , right = billaRES, how = "outer", on = "Index_bl")
df_bl

,Index_bl,name_bl,gtin
0,bl 0,Mautner Markhof Kräuteressig,9011900100135
1,bl 0,Mautner Markhof Kräuteressig,9011900105161
2,bl 1,Einmal Mund-Nasen Schnellmaske 3 Stk.,9002238270959
3,bl 1,Einmal Mund-Nasen Schnellmaske 3 Stk.,9120106122060
4,bl 1,Einmal Mund-Nasen Schnellmaske 3 Stk.,9002331609946
...,...,...,...
10680,bl 6746,Babywell Feuchte Waschlappen,9009695000631
10681,bl 6746,Babywell Feuchte Waschlappen,9009695001461
10682,bl 6747,BILLA Bio Dinkel-Biskotten,9010158017684
10683,bl 6748,Peddy Meisenringe auf Tasse,NaN


In [9]:
df_ofw=pd.read_csv('0 Preliminary.csv')


df_ofw = df_ofw.rename(columns={"Name":"name_ofw","GTIN":"gtin"})
df_ofw['Index_ofw'] ='ofw ' + df_ofw.index.astype(str)
df_ofw.head(10)


,Brand,name_ofw,gtin,Index_ofw
0,Panera Bread,Asian Chicken Salad,0000000000,ofw 0
1,"San Cassiano,Pronto®",Chocolat au lait,0000000000000,ofw 1
2,NaN,Vitória crackers,0000000000017,ofw 2
3,courte paille,moutarde au moût de raisin,0000000000100,ofw 3
4,NaN,Sauce Sweety chili 0%,0000000000123,ofw 4
5,NaN,Mendiants,0000000000291,ofw 5
6,NaN,Salade de carottes râpées,0000000000949,ofw 6
7,NaN,Fromage blanc aux myrtilles,0000000000970,ofw 7
8,CROUS,Solène céréales poulet,0000000001199,ofw 8
9,"Crous Resto',Crous",Tarte noix de coco,0000000001281,ofw 9


In [11]:
# Merging Billa with OFW with same gtin
df=pd.merge(left = df_bl , right = df_ofw, how = "outer", on = "gtin")
df=df.dropna()
len(df)

2044

In [14]:
# 2044 from comparison
# billa has around 3530 gtins
# so it seems plausible to use OFW
100*2044/3530

57.903682719546744